# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score
import pylab as plt

import warnings
warnings.filterwarnings('ignore')

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
7000


## Пишем свой класс для SVM

In [2]:
import numpy as np
from random import randint
import random


#np.random.seed(42)
#random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
        #return 1/(1 + np.exp(-self.f(x))) #sigmoid
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        return 1.0 * sum(self.w) / self.C

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])
        #return - answer * np.log(self.a(x)) #cross-entropy

    # hinge loss derivative
    def der_loss(self, x, answer):
        return -answer if answer * self.f(x) < 1 else 0
        #return -answer * (1 - self.a(x)) #cross-entropy

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        loss = []
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(1000):  
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 1 / np.sqrt(k+1)

            # w update
            self.w = self.w - step * (self.der_loss(x, y) * x) - step * self.der_reg()
            
            # w_0 update
            self.w0 = self.w0 - step * self.der_loss(x, y)
            
            #if k % 10 == 0:
            loss.append(self.loss(x, y))
            
        return loss

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [18]:
model = MySVM()
loss = model.fit(X_train, y_train)
print(model.w, model.w0)

[-1.09168012 -0.15596881 -0.83141103  3.99547873 -2.32941634  0.65128536
 -2.8361654  -0.10932428 -1.71782369 -1.9642651   0.24245682  0.22052652
  3.74473834  1.34847615 -1.66200395  0.95727335  0.31193437  1.7813076
 -0.10659956  1.0310023 ] 2.3202637243918733


In [19]:
predictions = model.predict(X_test)

In [20]:
print(predictions)

[0 1 1 ... 1 1 0]


In [21]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ... 1 0 0] 3000 1506


In [22]:
print(len(predictions), sum(predictions))

3000 2078


In [23]:
print(accuracy_score(y_test, predictions.round()))

0.704


In [24]:
accuracy = []
C = 1
while C <= 10000:
    model = MySVM(C)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy.append(accuracy_score(y_test, predictions.round()))
    C *= 10

In [25]:
np.round(accuracy, decimals=4)

array([0.524 , 0.6963, 0.7173, 0.7203, 0.686 ])

## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

In [26]:
from sklearn.svm import LinearSVC

SVC = LinearSVC(C = 10000, verbose=True, loss = 'hinge')

In [27]:
SVC.fit(X_train, y_train)

[LibLinear]

LinearSVC(C=10000, loss='hinge', verbose=True)

In [28]:
SVC.coef_

array([[-0.05323166, -0.01887402,  0.07835111,  0.11014207,  0.01394391,
         0.18200367, -0.01817534,  0.14522257,  0.01448139,  0.07380014,
         0.03673744,  0.09749883,  0.20703722,  0.00762198, -0.07627502,
         0.00911793,  0.06680716,  0.0863858 , -0.03972147,  0.08085836]])

In [29]:
pred = SVC.predict(X_test)

In [30]:
accuracy_score(y_test, pred)

0.6886666666666666

In [31]:
C = 1
SVC_accuracy = []
while C <= 10000:
    SVC = LinearSVC(C=C, max_iter = 10000, loss = 'hinge')
    SVC.fit(X_train, y_train)
    pred = SVC.predict(X_test)
    SVC_accuracy.append(accuracy_score(y_test, pred))
    C *= 10

In [32]:
np.round(SVC_accuracy, decimals=4)

array([0.7977, 0.7953, 0.7547, 0.6927, 0.7123])